In [ ]:
import numpy as np
from tqdm import tqdm

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
matplotlib.rc('font', size=16)
plt.rcParams['figure.figsize'] = (12.0, 10.0)    # resize plots

import pandas as pd

Leff was curve traced from figure 1c in : https://xecluster.lngs.infn.it/dokuwiki/doku.php?id=xenon:xenon1t:sim:notes:digangi:sensitivity-profile-likelihood-ratio

# Qy

In [ ]:
def nr_electron_yield(recoil_energies):
    """Qy from Chris' wimpstat, which is apparently taken from some unidentified C code (given semicolons at linebreaks)
    recoil_energies and cutoff_energy are in keV, answer is in number of electrons produced at the interaction point.
    This is apparently a variation on the Bezrukov Qy (looking at RichardW's notes you can see there are many other
    choices which can also be called "Bezrukov").
    """
#     cutoff_energy = config['nr_electron_yield_cutoff_energy']
#     beyond_cutoff = config['nr_electron_yield_behaviour_below_cutoff']
#     if cutoff_energy > 0 and beyond_cutoff == 'const':
#         recoil_energies = np.clip(recoil_energies, cutoff_energy, float('inf'))

    ep = recoil_energies * (1.05 * np.power(10.,-3.))
    k = 1.65e-1
    se = k * np.sqrt(ep)
    sn = np.log(1 + 1.1383 * 1.068 * ep)
    sn /= 2 *(1.068 * ep + 0.01321 * np.power((1.068 * ep),0.21226) + 0.19593*np.power((1.068*ep),0.5))
    ratio = 1.09
    g = 0.032
    b = 1.38e5

    squiggle = se / sn;

    Ni = (b * ep * squiggle)/(1 + ratio);
    Nq = (4./g) * np.log(1 + (g * Ni)/4.);

    answer = Nq/recoil_energies

#     if cutoff_energy > 0 and beyond_cutoff == 'zero':
#         answer[recoil_energies < cutoff_energy] = 0
    return answer


In [ ]:
e = np.logspace(0, 3, 100)
plt.plot(e, nr_electron_yield(e))
plt.xscale('log')
plt.xlabel('Energy (keV)')
plt.ylabel('Electron yield (electrons/keV)')

In [ ]:
pd.DataFrame({'kevnr': e, 'qy': nr_electron_yield(e)}).to_csv('qy_bezrukov.csv')

# ER photon yields

In [ ]:
npz = np.load('nest_ER_photon_yield.npz')    # Obtained using Bart's histogram ripper
data_at_voltage = sorted([(int(k.split(' ')[0]), v) for k, v in npz.items()])
for voltage, (x, y) in data_at_voltage:   
    x = 10**x
    with open('beta_photon_yield_nest_%dV.csv' % voltage, mode='w') as out:
        out.write("# NEST photon yield for betas at %d V\n")
        out.write("# Curve traced from figure 1, upper right, using Bart's histogramripper.\n")
        out.write("# Jelle, 25 July 2017.\n")
        pd.DataFrame({'kever': x, 'yield': y}).to_csv(out)
    plt.plot(x, y, label=voltage)
plt.legend(loc='upper left')
plt.xscale('log')